In [ ]:
import json

import numpy as np
import pandas as pd
import pymongo
import scipy
from bson import ObjectId, json_util
from pandas.io.json import json_normalize
from pymongo import MongoClient as Connection
from scipy import sparse, spatial
from scipy.spatial.distance import cdist, pdist, squareform
from sklearn.metrics.pairwise import cosine_similarity

#scientific notation disabled form smaller numbers
pd.options.display.float_format = '{:.5f}'.format

# alles resultate anzeigen und nicht nur das letzte
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

pd.set_option('expand_frame_repr', False)

In [58]:
import pandas as pd
import numpy as np
from IPython.display import display, HTML

CSS = """
.output {
    flex-direction: row;
}
"""

HTML('<style>{}</style>'.format(CSS))

In [ ]:

connection = Connection()
db = connection.recipe_db
input_data = db.recipes_test_copy
#data = pd.DataFrame(list(input_data.find()))

data = json.loads(json_util.dumps(input_data.find()))
norm_data = json_normalize(data)



#schmeiss alle zutaten raus, die weniger als 4 mal verwendet werden
# ingredients_unique = ingredients[ingredients.groupby('ingredients_id').ingredients_id.transform(len) > 4]

# drop duplicate by a column name
 
# ingredients_unique.drop_duplicates(['ingredients_id'], keep='last')

# ingredients_unique = ingredients_unique.drop(ingredients_unique.columns[[0, 3, 4]], axis=1)

# ingredients_unique['ingredients_id'].value_counts()



In [ ]:
ingredients = json_normalize(data, record_path='ingredients', meta='id', record_prefix='ingredients_', errors='ignore' )
ingredients = pd.DataFrame(ingredients)

In [ ]:
nutritions = json_normalize(data, record_path='nutritions',
                            meta=['id', 'prep_time', 'rating', 'rating_count', 'ready_in_time', 'review_count'])


In [ ]:
# data cleansing

# schmeiss alle zutaten raus, die weniger als 5 mal verwendet werden
ingredients_filt = ingredients.groupby('ingredients_id').filter(lambda x: len(x) >= 5)

# drop alls rows with ingredients_id == 0
ingredients_filt = ingredients_filt[ingredients_filt.ingredients_id != 0]

# drop all duplicate
ingredients_filt = ingredients_filt.drop_duplicates(keep='first')

ingredients.id.count()
ingredients.id.nunique()
ingredients_filt.id.count()
ingredients_filt.id.nunique()

In [ ]:
id_list = [215014, 8669, 16700, 16354, 12720, 8652, 8887, 51283, 45954, 213742, 14595, 219164, 16348, 143082, 8665, 11758, 223042, 236609, 8778, 65896, 24264, 11679, 141678, 9023]


recipe_db = pd.get_dummies(ingredients_filt['ingredients_id']).groupby(ingredients_filt['id']).apply(max)



In [ ]:
# recipe_db.index.nunique()
recipe_db.count()
nutritions.id.count()

In [ ]:
#filtere Rezepte aus den Metadaten anhand ihrerr Zutatenmenge raus. 
not_wanted_recipes = recipe_db[recipe_db.mask(recipe_db==0).count(axis=1) <= 4]
not_wanted_recipes
mask = nutritions['id'].isin(not_wanted_recipes.index.values)
nutritions_filt = nutritions.loc[~mask]
# nutritions_filt.id.nunique()
nutrition_db = nutritions_filt.pivot_table(
        index=['id'], 
         columns=['name'],
         values=['amount'],
).reset_index()

nutrition_db.set_index('id', inplace=True)
#remove multiindex 'amount'

nutrition_db.columns = nutrition_db.columns.droplevel(0)
# nutritions_filt.set_index('id', inplace=True)
nutrition_db = nutrition_db.dropna()
nutrition_db.index.nunique()

In [ ]:
recipe_db_filt = recipe_db[recipe_db.mask(recipe_db==0).count(axis=1) >= 5]

user_nutrition = nutrition_db[nutrition_db.index.isin(id_list)]
user_recipes = recipe_db_filt[recipe_db_filt.index.isin(id_list)]
# user_nutrition.columns = user_nutrition.columns.droplevel(0)
user_nutrition.index.nunique()
user_recipes.index.nunique()

In [ ]:
# remove recipes which are missing in nutrition df from the ingredients tecipes dataframe
mask = recipe_db_filt.index.isin(nutrition_db.index.values)
recipe_db = recipe_db_filt.loc[mask]

recipe_db

In [ ]:
##Euklidische Distanz - rezept zu rezept vergelich

result_array = cdist(user_recipes, recipe_db,'jaccard')
result_2 = pd.DataFrame(result_array, columns=recipe_db.index.values, index=user_recipes.index.values)
# result_2

result_10 = pd.DataFrame(result_2.sum(), columns=['jaccard_distance_sum'])
result_10 = result_10.sort_values(by='jaccard_distance_sum')

result_10

In [ ]:
out = result_10.hist(column='jaccard_distance_sum', bins=200)
out

In [ ]:
##Euklidische Distanz - rezept zu rezept vergelich
# minkowski(u, v, p=2, w=None)
# rec = []
# rec = {
#     "Calcium":[1],
#     "Calories":[1],
#     "Calories from Fat":[1],
#     "Carbohydrates":[1],
#     "Cholesterol":[1],
#     "Dietary Fiber":[1],
#     "Fat":[1],
#     "Folate":[1],
#     "Iron":[1],
#     "Magnesium":[1],
#     "Niacin Equivalents":[1],
#     "Potassium":[1],
#     "Protein":[1],
#     "Saturated Fat":[1],
#     "Sodium":[1],
#     "Sugars":[1],
#     "Thiamin":[1],
#     "Vitamin A - IU":[1],
#     "Vitamin B6":[1],
#     "Vitamin C":[1]}

rec_2 = [
    #"Calcium":[1],
    1,
   # "Calories":[1],
    1,
   # "Calories from Fat":[1],
    1,
    #"Carbohydrates":[1],
    1,
    #"Cholesterol":[1],
    1,
    #"Dietary Fiber	":[1],
    1,
    #"Fat":[1],
    1,
    #"Folate":[1],
    1,
    #"Iron":[1],
    1,
    #"Magnesium":[1],
    1,
    #"Niacin Equivalents":[1],
    1,
    #"Potassium":[1],
    1,
    #"Protein":[1],
    1,
    #"Saturated Fat":[1],
    1,
    #"Sodium":[1],
    1,
    #"Sugars":[1],
    1,
    #"Thiamin":[1],
    1,
    #"Vitamin A - IU":[1],
    1,
    #"Vitamin B6":[1],
    1,
    #"Vitamin C":[1]}
    1]

# nutrition_db = nutrition_db[:-700]
# nutrition_db = nutrition_db.dropna()  
result_array = cdist(user_nutrition, nutrition_db,'minkowski', p = 2, w= rec_2)
result_2 = pd.DataFrame(result_array, columns=nutrition_db.index.values, index=user_nutrition.index.values)
# result_2

result_6 = pd.DataFrame(result_2.sum(), columns=['minkowski_distance_sum'])
result_6 = result_6.sort_values(by='minkowski_distance_sum')


In [ ]:
test = result_6
test2 = result_10
# result_6.describe()
# out = result_6.hist(column='minkowski_distance_sum', bins=2000)
# out

In [ ]:
result_6 = result_6[:-700]


In [ ]:
# result_6.describe()
out1 = result_6.hist(column='minkowski_distance_sum', bins=200)
out2 = result_10.hist(column='jaccard_distance_sum', bins=200)
out1
out2

In [ ]:
outliers = test[test['minkowski_distance_sum'] > test['minkowski_distance_sum'].mean() + 1 * test['minkowski_distance_sum'].std()]


In [ ]:
test.drop(outliers.index, inplace=True)

In [ ]:
test.describe()
test2.describe()

In [ ]:
test2 = test2[test2.index.isin(test.index)]
test2

In [ ]:
import pylab
pylab.scatter(test2.jaccard_distance_sum, test.minkowski_distance_sum)

In [ ]:

# outliers

# # test2.drop(outliers.index, inplace=True)
# test2
result_6 = test
result_10 = test2

In [ ]:
test1 = test.hist(column='minkowski_distance_sum', bins=200)
test1

In [ ]:
result_6 = test

In [ ]:
#minmax nomralisierung
from sklearn import preprocessing

nut_res = result_6
x = nut_res.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
# jac_res = jac_res.reset_index()
nut_res = pd.DataFrame(x_scaled, columns=nut_res.columns, index=nut_res.index)
# result_6 = pd.DataFrame(x_scaled, columns=result_6.columns)


jac_res = result_10
z = jac_res.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
z_scaled = min_max_scaler.fit_transform(z)
# jac_res = jac_res.reset_index()
jac_res = pd.DataFrame(z_scaled, columns=jac_res.columns, index=jac_res.index)

min_max_nut_recipe = pd.DataFrame(nut_res.minkowski_distance_sum + jac_res.jaccard_distance_sum, columns=jac_res.columns, index=jac_res.index)
min_max_nut_recipe = min_max_nut_recipe.sort_values(by='jaccard_distance_sum')
min_max_nut_recipe.rename(columns={'jaccard_distance_sum': 'min_max_nut_recipe'}, inplace=True)

In [ ]:
out1 = nut_res.hist(column='minkowski_distance_sum', bins=200)
out2 = jac_res.hist(column='jaccard_distance_sum', bins=200)
out1
out2

In [ ]:
display_side_by_side(jac_res, nut_res, min_max_nut_recipe)

In [ ]:
display(jac_res, nut_res, min_max_nut_recipe)

In [ ]:
#robust-normalisierung
from sklearn import preprocessing

nut_res_robust = result_6
x = nut_res_robust.values #returns a numpy array
robust_scaler = preprocessing.RobustScaler()
x_scaled = robust_scaler.fit_transform(x)
# jac_res = jac_res.reset_index()
nut_res_robust = pd.DataFrame(x_scaled, columns=nut_res_robust.columns, index=nut_res_robust.index)
# result_6 = pd.DataFrame(x_scaled, columns=result_6.columns)


jac_res_robust = result_10
z = jac_res_robust.values #returns a numpy array
robust_scaler = preprocessing.RobustScaler()
z_scaled = robust_scaler.fit_transform(z)
# jac_res_robust = jac_res_robust.reset_index()
jac_res_robust = pd.DataFrame(z_scaled, columns=jac_res_robust.columns, index=jac_res_robust.index)



robust_scaling_nut_recipe = pd.DataFrame(nut_res_robust.minkowski_distance_sum + jac_res_robust.jaccard_distance_sum, columns=jac_res_robust.columns, index=jac_res_robust.index)
robust_scaling_nut_recipe = robust_scaling_nut_recipe.sort_values(by='jaccard_distance_sum')
robust_scaling_nut_recipe.rename(columns={'jaccard_distance_sum': 'robust_scaling_nut_recipe'}, inplace=True)

In [ ]:
out1 = nut_res_robust.hist(column='minkowski_distance_sum', bins=200)
out2 = jac_res_robust.hist(column='jaccard_distance_sum', bins=200)
out1
out2

In [ ]:
display_side_by_side(jac_res_robust, nut_res_robust)

In [ ]:
#z-score-mormalisierung
z_min = (result_6 - result_6.mean())/result_6.std()
z_jac = (result_10 - result_10.mean())/result_10.std()

z_score_nut_recipe = pd.DataFrame(z_min.minkowski_distance_sum + z_jac.jaccard_distance_sum, columns=result_6.columns, index=z_min.index)
z_score_nut_recipe = z_score_nut_recipe.sort_values(by='minkowski_distance_sum')
z_score_nut_recipe.rename(columns={'minkowski_distance_sum': 'z_score_nut_recipe'}, inplace=True)

In [ ]:
out1 = result_6.hist(column='minkowski_distance_sum', bins=200)
out2 = result_10.hist(column='jaccard_distance_sum', bins=200)
out1
out2

In [ ]:
display_side_by_side(result_6, result_10)

In [ ]:
z_scaled

In [ ]:
# summed_up = pd.DataFrame(columns='sum')
# result_6['sum'] = result_6.minkowski_distance_sum + result_10.jaccard_distance_sum
# result_6


In [59]:
display(result_6, result_10)

,minkowski_distance_sum
72845,16845.99911
180791,16895.05504
213742,16972.68289
8621,16977.63899
217080,17040.46949
219461,17052.20530
90497,17061.19406
223381,17116.44969
25198,17168.34636
162936,17177.05649


,jaccard_distance_sum
102235,19.33806
11732,19.37608
25927,19.45397
16416,19.53403
8887,19.57872
51372,19.59542
223042,19.60473
132511,19.68490
213487,19.69363
14534,19.70749


In [57]:
display(result_6, result_10, min_max_nut_recipe, z_score_nut_recipe, robust_scaling_nut_recipe)


,minkowski_distance_sum
72845,16845.99911
180791,16895.05504
213742,16972.68289
8621,16977.63899
217080,17040.46949
219461,17052.20530
90497,17061.19406
223381,17116.44969
25198,17168.34636
162936,17177.05649


,jaccard_distance_sum
102235,19.33806
11732,19.37608
25927,19.45397
16416,19.53403
8887,19.57872
51372,19.59542
223042,19.60473
132511,19.68490
213487,19.69363
14534,19.70749


,min_max_nut_recipe
25927,0.05174
102235,0.08032
16416,0.08062
51372,0.08277
8887,0.08906
223042,0.09609
11732,0.12064
213487,0.14029
19126,0.15564
24264,0.15912


,z_score_nut_recipe
25927,-3.94362
102235,-3.78660
16416,-3.77637
51372,-3.76155
8887,-3.72658
223042,-3.68561
11732,-3.55630
213487,-3.43109
19126,-3.33799
24264,-3.32179


,robust_scaling_nut_recipe
25927,-2.87850
51372,-2.78256
16416,-2.72670
8887,-2.70813
223042,-2.67911
102235,-2.49205
19126,-2.44256
86047,-2.42698
213487,-2.40605
238038,-2.37094


In [ ]:
robust_scaling_nut_recipe.index.values[:10]

In [ ]:
user_nutrition.describe()

In [55]:
display(jac_res_robust, nut_res_robust, robust_scaling_nut_recipe)

,jaccard_distance_sum
102235,-2.43501
11732,-2.39163
25927,-2.30274
16416,-2.21138
8887,-2.16038
51372,-2.14133
223042,-2.13070
132511,-2.03921
213487,-2.02925
14534,-2.01344


,minkowski_distance_sum
72845,-0.74880
180791,-0.74404
213742,-0.73651
8621,-0.73603
217080,-0.72993
219461,-0.72879
90497,-0.72792
223381,-0.72256
25198,-0.71752
162936,-0.71668


,robust_scaling_nut_recipe
25927,-2.87850
51372,-2.78256
16416,-2.72670
8887,-2.70813
223042,-2.67911
102235,-2.49205
19126,-2.44256
86047,-2.42698
213487,-2.40605
238038,-2.37094


In [ ]:
import matplotlib.pyplot as plt

df.plot(kind='scatter',x='year',y='value')
plt.show()

In [ ]:
res[:100]

In [ ]:
result_6[:10].index.values

In [ ]:
# nutrition_db.isnull().values.sum()
# nutrition_db
# df1 = nutrition_db[nutrition_db.isnull().any(axis=1)]

# df1
# result_6['minkowski_distance_sum']
user_nutrition.describe()

In [ ]:
top_10_nut = nutrition_db[nutrition_db.index.isin(result_6.index.values[:10])]
top_10_nut_recipes = nutrition_db[nutrition_db.index.isin(robust_scaling_nut_recipe.index.values[:10])]
top_10_recipes = nutrition_db[nutrition_db.index.isin(result_10.index.values[:10])]

In [ ]:
top_10_nut_recipes.describe()


In [ ]:
nutrition_db.loc[[24717]]

In [ ]:
user_nutrition.describe()
top_10_recipes.describe()
top_10_nut.describe()

In [ ]:
# user_nutrition['Vitamin B6'].median()
user_nutrition['Fat'].median()

In [ ]:
#remove multiindex 'amount'
# user_nutrition.columns = user_nutrition.columns.droplevel(0)
# nutrition_db.columns = nutrition_db.columns.droplevel(0)